## Kryptografia w bezpieczeństwie sieci bezprzewodowych 
#### Adrianna Kralka, Jakub Młocek, Marcel Trzaskawka 

### Spis treści
- Teoria	
    - Kryptografia w sieciach	
    - WEP Wired Equivalent Privacy	
        - Opis:	
        - Szyfr RC4:	
        - Suma Kontrolna CRC-32:	
        - Wykryte podatności:	
        - Wnioski:	
- WPS Wifi protected setup	
- Bluetooth	
- WPA
- Standard przejściowy między WEP, a WPA2. EAP, Temporal Key Integruty      Protocol (TKIP), RC4,i Messege Integrity Check (Michael algorithm)	
- WPA2	
    - CCMP
    - Wykryta podatność:	
    - Praktyczne ukazanie ataku KRACK na WPA2:	
    - Wnioski:	
- WPA3
- CCMP/AES (WPA3-Personal)
- GCM/AES (WPA3-Enterprise
- WPA-PSK zastąpiony Simultaneous Authentication of Equals (SAE)	
- Źródła


## WEP Wired Equivalent Privacy
- Opis:
Standard Wired Equivalent Privacy (WEP) został wprowadzony w 1999 roku jako część oryginalnego standardu 802.11. Miało to na celu zapewnienie poufności na poziomie zbliżonym do sieci kablowych. W standardzie używany jest szyfr strumieniowy RC4 który miał zapewniać zachowanie poufności poufności oraz suma kontrolna CRC-32 dla zapewnienia integralności. Szyfrowanie występuje w dwóch wersjach z różną długością klucza RC4: 64 i 128 bitów. <br>
<br>
- Szyfr RC4:
RC4 jest szyfrem strumieniowym. Szyfrowanie za pomocą RC4 polega na sumowaniu XOR kolejnych bajtów wiadomości z kolejnymi bajtami strumienia klucza. Szczegóły działania szyfru:

1. Inicjowanie tabeli T - Tabela T ma długość 256 bajtów. Jest wypełniana bajtami na podstawie wartości sekretnego klucza, przed rozpoczęciem operacji szyfrowania lub deszyfrowania. Następujące operacje powinny zostać wykonane, aby utworzyć tabelę:
- W każdą komórkę tablicy wpisywana jest liczba odpowiadająca numerowi jej pozycji (licząc od zera).
- Tworzona jest zmienna pomocnicza i inicjowana na 0.
- Dla każdej liczby w tablicy (numery przyjmują wartości kolejno od 0 do 255) wykonywane są dwie operacje: <br>
     <br>   1.1. Wartość zmiennej pomocniczej jest modyfikowana zgodnie ze wzorem:
-              X = (X + W + K) mod 256, gdzie:
     - X - zmienna pomocnicza,
     - W - wartość aktualnej pozycji w tabeli T,
     - K - wartość bajtu klucza w aktualnej pozycji podzielonej modulo przez długość klucza (ponieważ sekretny klucz może być krótszy niż 256 bajtów),

     1.2.Liczba w tablicy w aktualnej pozycji jest zamieniana miejscem z liczbą w tabeli w pozycji określonej przez zmienną pomocniczą.

2. Szyfrowanie i Deszyfrowanie - W trakcie szyfrowania i deszyfrowania generowane są bajty strumienia klucza, które następnie sumuje się XOR z bajtami wiadomości Bajty strumienia klucza produkowane są na podstawie tablicy T. Następujące kroki są wykonywane:

- Tworzone są dwie zmienne pomocnicze p1 i p2, obie inicjowane są na 0.
- Do zmiennej p1 dodaje się 1, a wynik dzieli modulo przez 256.
- Do zmiennej p2 dodaje się wartość tablicy na pozycji p1 (czyli T[p1]), a wynik dzieli modulo przez 256.
- Zamienia się miejscami wartości tablicy na pozycjach p1 i p2.
- Sumuje się dwie wartości tablicy na pozycjach p1 i p2, a wynik dzieli modulo przez 256. Następnie wynik przypisuje się do nowej zmiennej pomocniczej p3.
- Wartość tablicy na pozycji p3 jest kolejnym bajtem strumienia klucza.
- Jeśli potrzebne są kolejne bajty strumienia klucza, to należy wrócić do punktu II i powtórzyć powyższe kroki.
<br>
<br> Implementacja szyfru rc4 w języku python  na podstawie powyższych kroków:



In [ ]:
import codec
SIZE = 256
def createKeyTable(key):
   T = list(range(SIZE))
   pom = 0
   for i in range(SIZE):
       pom = (pom + T[i] + key[i % len(key)]) % SIZE
       T[i], T[pom] = T[pom], T[i]
   return T

def keyStreamGenerator(T):
   p1 = 0
   p2 = 0
   while True:
       p1 = (p1 + 1) % SIZE
       p2 = (p2 + T[p1]) % SIZE
       T[p1], T[p2] = T[p2], T[p1]
       p3 = (T[p1] + T[p2]) % SIZE
       yield p3

def getKeyStream(key):
   T = createKeyTable(key)
   return keyStreamGenerator(T)

def encrypt(key, plaintext):
   key = [ord(c) for c in key]
   if type(plaintext) == str:
       plaintext = [ord(c) for c in plaintext]
   keystream = getKeyStream(key)
   res = []
   for c in plaintext:
       val = ("%02X" % (c ^ next(keystream))) 
       res.append(val)
   return ''.join(res)


def decrypt(key, ciphertext):
   ciphertext = codecs.decode(ciphertext, 'hex_codec')
   res = encrypt(key, ciphertext)
   return codecs.decode(res, 'hex_codec').decode('utf-8')


if __name__ == "__main__":
   key = "zajprojektowezkryptografii"
   plaintext = "Mozeudanamsiedostac5zprojketu"
   ciphertext = encrypt(key,plaintext)
   print(f"plaintex: {plaintext}\nciphertext: {ciphertext}\n key: {key}")
   ciphertext = "8AEE24852B79ED2B6BC6FD005EE43344432D4889335E4DF55E4CB0F369"
   decryptedPlaintext = decrypt(key,ciphertext)
   print(f"decrytped plain text: {decryptedPlaintext}")

### Suma Kontrolna CRC-32:
Algorytm CRC (Cyclic Redundancny Check) 32bit służy do wyznaczania sum kontrolnych dla dowolnych danych wejściowych. Jest to nic innego jak 32-bitowa liczba całkowita określająca poprawność danych ze wzorcem. 
Podstawą działania algorytmu CRC-32 jest wielomian:
x^32+x^26+x^23+x^22+x^16+x^12+x^11+x^10+x^8+x^7+x^5+x^4+x^2+x^1+x^0
Wielomian ten jest reprezentowany jako pojedyncza liczba 32-bitowa. Każdy fragment
to bit o wartości 1 ustawiony w miejscu wyznaczonym przez wykładnik potęgi:
100000100110000010001100110110111 (2)
Jej długość wynosi 33 bity więc o jeden bit za dużo. Usuwamy najstarszy bit i otrzymujemy 32-bitową reprezentację wielomianu CRC-32.
00000100110000010001100110110111 (2) = 04C11DB7 (16) 
Wartość ta jest zawsze stała. <br>
<br> 
Następnie dla kazdego kodu ASCII obliczamy wartości:



In [ ]:
for(ascii = 0; ascii < 255; ascii = ascii + 1)   
   A[ascii] = ascii;
   odwróć 8 najmłodszych bitów A[ascii];
   A[ascii] = A[ascii] << 24;                       # przesuń o 24 bity w lewo
   for(i = 0; i < 8; i = i + 1)                     # petla od 0 do 7
     if((A[ascii] AND 0x80000000) != 0)             # jesli jest rożne od 0
       A[ascii] = (A[ascii] << 1) XOR 0x04C11DB7;   # przesuń o 1 bit w lewo i xor'uj z 0x04C11DB7
     else
       A[ascii] = A[ascii] << 1;                    # przesuń o 1 bit w lewo
   odwróć 32 bity A[ascii];

   #Wartości wyliczone w powyższym algorytmie są zazwyczaj obliczane raz 
   # i podawane jako stała. 
   # Teraz możemy już obliczyć sumę kontrolną dla dowolnego ciągu danych:

crc32 = 0xFFFFFFFF;   // wartość początkowa
for(i = 0; i < n; i = i + 1)   // pętla od 0 do n - 1
   crc32 = (crc32 >> 8) XOR A[(crc32 AND 0xFF) XOR B[i]];
crc32 = crc32 XOR 0xFFFFFFFF;

+ Wykryte podatności: <br>
<br>
W standarcie WEP wykorzystano wektor inicjalizacyjny, który konkatenowany z kluczem podanym przez użytkownika miał zabezpieczać przed transmisją danych z użyciem tego samego klucza RC4. Ponieważ jest on zmienny, strona, która wysyła dane i wybiera wartość tego wektora, musi ją przekazać odbiorcy, aby umożliwić odszyfrowanie danych. W związku z tym wektor inicjalizacyjny musi być transmitowany w nie zaszyfrowanej części pakietu. W praktyce okazało się, że w sieciach z dużym ruchem 24-bitowa przestrzeń wartości wektora inicjalizacyjnego jest niewystarczająca, gdyż prawdopodobieństwo wystąpienia kolizji polegającej na powtórzeniu się wektora inicjalizującego wynosi 50% już po odebraniu 5000 pakietów. W przypadku powtórzenia się wektora inicjalizujacego dane są szyfrowanie takim samym kluczem. Otworzyło to drogę do ataku określanego jako „related key”, wykorzystującego podobieństwa i zależności matematyczne dotyczące użytych kluczy. W latach 2001–2007 opublikowano wiele prac dotyczących kryptoanalizy i specyficznych własności RC4, dających ostatecznie możliwość odtworzenia klucza tylko z wykorzystaniem pasywnie podsłuchanej komunikacji sieciowej. W 2007 opisano metodę, która jest w stanie odtworzyć 104-bitowy klucz z prawdopodobieństwem 50% po uzyskaniu 40k pakietów. Dla 60k pakietów natomiast prawdopodobieństwo to wynosi już 80%, a dla 85k pakietów – aż 95%! W przypadku klucza 40-bitowego nawet 20k pakietów może wystarczyć do odzyskania klucza. Ponadto przy pomocy technik takich jak odłączenie klienta i wstrzykiwanie pakietów, uzyskanie 40k pakietów może zająć czas poniżej minuty! Atakującemu, który posiada wymaganą liczbę pakietów, złamanie klucza zajmuje raptem kilka sekund. Aby wytworzyć sztuczne pakiety  stworzono technikę określaną jako wstrzykiwanie pakietów, polegającą na wysyłaniu do sieci nie odszyfrowanych pakietów w taki sposób, aby wymusić generowanie nowych wektorów inicjalizacyjnych. Wykorzystane zostały do tego żądania protokołu ARP, które są wysyłane na adres rozgłoszeniowy, mają stałą długość 68 bajtów i często pojawiają się w sieci. Z tego względu można je łatwo wykryć, przechwycić i ponownie przesłać do sieci, bez konieczności rozszyfrowywania. Każde żądanie jest rozgłaszane przez AP i otrzymuje nowy wektor inicjalizacyjny. Ten sam pakiet można wysyłać dowolną liczbę razy, zwiększając znacząco liczbę przechwyconych wektorów.

+ Wnioski: <br>
<br>
W związku z pojawieniem się sieci bezprzewodowych, pojawiła się konieczność ich zabezpieczenia. Naprzeciw temu szybko powstał standard szyfrowania WEP. Niestety standard ten zawiera błędy które pozwalają w łatwy sposób łamać klucze WEP. Aktualnie jest to standard nieużywany oraz niezalecany, a jego rolę przejęli jego następcy.



#### WPS Wifi protected setup

Opis:
- Wifi protected setup (WPS) to standard bezpieczeństwa sieci bezprzewodowej, który służy do tworzenia chronionej i bezpiecznej bezprzewodowej sieci domowej. WPS próbuje nawiązywać połączenia między routerem, a urządzeniami bezprzewodowymi w szybszy i łatwiejszy sposób. 
Funkcja WPS działa tylko w przypadku sieci bezprzewodowych korzystających z hasła chronionego za pomocą protokołów bezpieczeństwa WPA lub WPA2 i tylko w najnowszych wersjach routera.
Nie działa natomiast w sieciach bezprzewodowych, które korzystają z zabezpieczeń WEP, czyli takich, które mogą łatwo zostać złamane.

- W standardowej konfiguracji: nie można podłączyć urządzenia do sieci bezprzewodowej, dopóki nie zna się nazwy sieci i hasła. Chcąc podłączyć urządzenie, należy wybrać sieć, z którą chce się połączyć, a następnie wprowadzić jej hasło zabezpieczające.

- WPS:  może uprościć proces łączenia na następujące sposoby:
1. Należy wcisnąć przycisk WPS na routerze, aby   wykrywać nowe urządzenia. Wybrać sieć, z którą chcemy połączyć się na naszym urządzeniu, a ono automatycznie połączy się z siecią bezprzewodową bez podawania hasła sieciowego.
2. Dla takich urządzeń jak np. drukarki bezprzewodowe, wystarczy wcisnąć przycisk WPS na obu urządzeniach i bez podawania żadnych danych WPS automatycznie wysyła hasło sieciowe, które te urządzenia zapamiętują do wykorzystania w przyszłości. W przyszłości urządzenia będą mogły łączyć się z tą samą siecią bez ponownego używania przycisku WPS.
3. Niektóre urządzenia nie mają przycisku WPS, ale wszystkie routery z włączonym WPS mają generowany automatycznie, ośmiocyfrowy kod PIN. Jeśli kod PIN zostanie wprowadzony, nastąpi uwierzytelnienie i połączenie z siecią.
4. Niektóre urządzenia bez przycisku WPS, ale z obsługą WPS generują również kod PIN klienta. Można wprowadzić go  w panelu konfiguracji bezprzewodowej routera, a następnie router użyje go, aby dodać dane urządzenie do sieci.

Metoda 3 i 4 nie są jednak zbyt optymalne czasowo, w przeciwieństwie do metody 1 oraz 2.

- Zalety WPS:
    - Klucz bezpieczeństwa SSID(Service Set Identifier) lub hasło nie muszą być znane
    - WPS automatycznie konfiguruje nazwę sieci (SSID) i klucz bezpieczeństwa WPA dla punktu dostępowego z włączonym WPS w sieci.
    - Klucze WPS są generowane losowo
    - WPS umożliwia właścicielowi uprawnień Wi-Fi blokowanie innym użytkownikom możliwości korzystania z tej sieci.
    - WPS ma dwa tryby uwierzytelniania PBC, tj. Push Button Configuration i PIN, czyli osobisty numer identyfikacyjny.

- Wady WPS:
    - Trudno połączyć klienta innego niż WPS ze względu na długie sekwencje znaków szesnastkowych.
    - Może być używany tylko w urządzeniach obsługujących WPS, w przeciwnym razie musimy nawiązać połączenie w tradycyjny sposób.
    - PIN jest mało bezpieczny, ponieważ jest to kod numeryczny, który można łatwo złamać przy użyciu Brute-force’a.




### Bluetooth 

Opis:
- Bluetooth  to technologia transmisji radiowej krótkiego zasięgu i małej mocy. Składa się z niejonizujących fal elektromagnetycznych w zakresie radiowym między 2.402 GHz, a 2.48 GHz. Będąc na dolnym końcu zakresu, Bluetooth jest energooszczędny w porównaniu z innymi urządzeniami do transmisji radiowej. Dlatego łatwo jest go zaimplementować w małych gadżetach i smartfonach, nie martwiąc się zbytnio o żywotność baterii.

- Bluetooth może podróżować po otwartych przestrzeniach i przez niektóre bariery bez fizycznego nośnika. To, jak daleko może dotrzeć określony sygnał Bluetooth, zależy przede wszystkim od jego siły oraz od tego, czy bariery są wystarczająco grube lub wykonane z materiałów, które zniekształcają i degradują sygnał nie do poznania.
- Sygnał Bluetooth wysyłany przez telefon do zestawu słuchawkowego jest najpierw poprzedzony przez oprogramowanie. Dane — zdjęcia, dźwięk i tak dalej — nie mogą być przesyłane przez Bluetooth. Najpierw przechodzi przez algorytm kompresji zwany kodekiem, który przekształca dane w pojedyncze pakiety.

- Kodeki, takie jak aptX, AAC, SBC i LDAC, muszą być zgodne między urządzeniami nadawczymi i odbiorczymi. Różne kodeki są optymalne dla różnych typów danych i wpływają na takie czynniki, jak zużycie energii, szybkość przesyłania danych, jakość danych i zasięg połączenia.


- Kodek Bluetooth to program umożliwiający bezprzewodową transmisję między źródłem dźwięku, a odtwarzaczem audio. Koduje dane odebrane ze źródła i przesyła je do kompatybilnego odtwarzacza, gdzie ten sam kodek dekoduje te dane.

- Mówiąc najprościej, jest to kanał, przez który dźwięk przechodzi z telefonu do słuchawek Bluetooth. Celem jest przesyłanie dźwięku z minimalną utratą danych i minimalnymi opóźnieniami — naśladując jakość dźwięku przewodowego.

